# RUN SCC-DFTB+CCS+PiNN Simulations using ASE 
Input: SKF/, CCS/ and PiNN/ folders from ParAutomatik

Simualtion type: 
- Geometry optimization
- MD Simulations 
- and more 

In [1]:
PARAUTOMATIK_PATH='/Users/tjams20/Documents/repos/ParAutomatik/'   # Add your installation path here

In [2]:
import os 
import sys
from ase import Atoms
from ase import io
from ase.optimize import FIRE
from ase.io import read, write
from ase.calculators.dftb import Dftb
from ase.calculators.mixing import LinearCombinationCalculator
import json
# from ase.calculators.ccs import CCS
# from ccs.ase_calculator.ccs_ase_calculator import CCS # PATH ADDED USING conda develop
import numpy as np
# import pinn
# print(dir(pinn))
# from xtb.ase.calculator import XTB
sys.path.append(PARAUTOMATIK_PATH+'/pgm/CCS/')

base_dir=os.getcwd()
os.chdir(base_dir)

if not os.path.isdir("RUN_SIM/"):
    os.mkdir("RUN_SIM/")

os.chdir("RUN_SIM/")
cwd = os.getcwd() 
print('You are here',cwd) 

def setup_model(elements,spin,charge,**kwargs):

    # set up model:
    # 1: DFTB 
    os.environ["DFTB_PREFIX"] = base_dir+"/SKF/"
    DFTB_calc=Dftb(Hamiltonian_SCC='Yes',Hamiltonian_ShellResolvedSCC = 'Yes',Hamiltonian_SCCTolerance = '1.0E-006',Hamiltonian_Mixer= 'Anderson{',Hamiltonian_Mixer_MixingParameter='0.03',Hamiltonian_MaxSCCIterations = '1500',Hamiltonian_Filling = 'MethfesselPaxton{',Hamiltonian_Filling_Temperature = '0.001583407672623195',Hamiltonian_KPointsAndWeights = 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}') 
    if spin>0:                  
        DFTB_calc.set(Hamiltonian_SpinPolarisation = 'Colinear{') 
        DFTB_calc.set(Hamiltonian_SpinPolarisation_UnpairedElectrons=str(spin))
        DFTB_calc.set(Hamiltonian_SpinConstants_ ='')
        for i in elements: 
            print(i)
            spinparam=get_spindata(i)
            string=f'Hamiltonian_SpinConstants_{i}'  
            print(string)
            DFTB_calc.set(**{string:spinparam})  
    if charge>0: 
        DFTB_calc.set(Hamiltonian_Charge =str(charge))
    print(DFTB_calc.todict())     

    
    # 1.1 DFTB with xTB
    # XTB_calc = XTB(method="GFN1-xTB")
    DFTB_calc=Dftb(label='singlepoint', Hamiltonian_='xTB', Hamiltonian_Method='GFN2-xTB', 
                   Hamiltonian_ReadInitialCharges='Yes', # Hamiltonian_ParameterFile=skdef_all,
                   Hamiltonian_MaxSCCIterations = '1500', Hamiltonian_Filling = 'MethfesselPaxton{',
                   Hamiltonian_Filling_Temperature = '0.003583407672623195',Hamiltonian_Mixer= 'Anderson{',
                   Hamiltonian_Mixer_MixingParameter='0.03', Hamiltonian_SCCTolerance = '3.0E-003',
                   Hamiltonian_KPointsAndWeights = 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}',
                   Hamiltonian_MaxAngularMomentum_Li="d", Hamiltonian_MaxAngularMomentum_C="d",
                   Hamiltonian_MaxAngularMomentum_O="d", Hamiltonian_MaxAngularMomentum_H="s",
                   do_forces = True, write_results = True)
    
    #2: CCS 
    # with open(base_dir+'/CCS/CCS_params.json', 'r') as f:
    #     CCS_params = json.load(f)

    # CCS_calc=CCS(CCS_params=CCS_params)
    
    #3: PiNN 
    # PiNN_calc = pinn.get_calc(base_dir+'/PiNN/') 
    

    # and the calculater becomes 
    calcs =[DFTB_calc] #, CCS_calc] #, PiNN_calc]
    weights=[1] #,1] #,1]
    calc=LinearCombinationCalculator(calcs, weights) 
    return calc            

def get_spindata(element): 
    if element=='Li':
        spinparam='{-0.019  -0.016  -0.016 -0.027}'
    if element=='C':
        spinparam='{-0.031 -0.025 -0.025 -0.023}'
    if element=='O':
        spinparam='{-0.035 -0.030 -0.030 -0.028}'
    if element=='H':
        spinparam='{-0.072}'
    return spinparam        

You are here /Users/tjams20/Documents/repos/ParAutomatik/pgm/Jupyter_notebooks/RUN_SIM


In [3]:
import ase
print(ase.__path__)

['/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/ase']


In [4]:
# Geometry optimization
# import structure:
slab = io.read('/Users/tjams20/Documents/repos/ParAutomatik/Electrolyte/DFT_DB.db@0')
# slab = io.read('../DFT_DB.db@0')
elements=['C','H','O'] 
spin=0
charge=0
slab.calc=setup_model(elements, spin, charge)
nrg = slab.get_potential_energy()
relax = FIRE(slab, trajectory='opt.traj')
relax.run(fmax=0.05) 

{'Hamiltonian_SCC': 'Yes', 'Hamiltonian_ShellResolvedSCC': 'Yes', 'Hamiltonian_SCCTolerance': '1.0E-006', 'Hamiltonian_Mixer': 'Anderson{', 'Hamiltonian_Mixer_MixingParameter': '0.03', 'Hamiltonian_MaxSCCIterations': '1500', 'Hamiltonian_Filling': 'MethfesselPaxton{', 'Hamiltonian_Filling_Temperature': '0.001583407672623195', 'Hamiltonian_KPointsAndWeights': 'SupercellFolding {1 0 0 0 1 0 0 0 1 0.0 0.0 0.0}'}
      Step     Time          Energy         fmax
FIRE:    0 15:07:42     -689.785540        3.4170
FIRE:    1 15:07:42     -690.243724        2.2679
FIRE:    2 15:07:42     -690.528573        2.5506
FIRE:    3 15:07:42     -690.569470        1.7876
FIRE:    4 15:07:42     -690.623513        1.3493
FIRE:    5 15:07:42     -690.670127        1.2242
FIRE:    6 15:07:42     -690.712895        1.2804
FIRE:    7 15:07:42     -690.763702        1.4141
FIRE:    8 15:07:42     -690.823751        1.1140
FIRE:    9 15:07:42     -690.883187        0.8313
FIRE:   10 15:07:42     -690.936106   

True

In [5]:
!ase gui opt.traj

Traceback (most recent call last):
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/ase/gui/pipe.py", line 28, in main
    plt.show()
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/matplotlib/pyplot.py", line 411, in show
    return _get_backend_mod().show(*args, **kwargs)
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/matplotlib_inline/backend_inline.py", line 90, in show
    display(
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-p

In [6]:
# Read geometry optimized structure and run NVE Molecular dynamics 
from ase import units
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.md import MDLogger
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.nptberendsen import NPTBerendsen
from ase.md.nvtberendsen import NVTBerendsen
from ase.md.verlet import VelocityVerlet

rng = np.random.default_rng(10)
atoms = read("opt.traj")
atoms.set_calculator(slab.calc)
MaxwellBoltzmannDistribution(atoms, 873*units.kB, rng=rng)
dt = 0.5 * units.fs
steps = int(10)
dyn = VelocityVerlet(atoms, timestep=dt)
interval = int(1)
dyn.attach(MDLogger(dyn, atoms, 'output.log', mode="a"), interval=interval)
dyn.attach(Trajectory('output.traj', 'a', atoms).write, interval=interval)
dyn.run(steps)

/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/ase/md/md.py:48: FutureWarning: Specify the temperature in K using the 'temperature_K' argument
  warnings.warn(FutureWarning(w))


True

In [7]:
!ase gui output.traj 

Traceback (most recent call last):
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/ase/gui/pipe.py", line 32, in <module>
    main()
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/ase/gui/pipe.py", line 28, in main
    plt.show()
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/matplotlib/pyplot.py", line 411, in show
    return _get_backend_mod().show(*args, **kwargs)
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-packages/matplotlib_inline/backend_inline.py", line 90, in show
    display(
  File "/Users/tjams20/opt/anaconda3/envs/ParAutomatik/lib/python3.10/site-p